# Calculations of the Effect Size (ES) for each microarray study 

###  Using Hedges' g value, an adjusted Cohen's d  value

$$  {Enrichment} = \bar{X_2}-\bar{X_1}$$

Let Group 1 be 9h Sleeping Hypothalamus Expression values and Group 2 be 9h SD Hypothalamus Expression values 

(S mean - SD mean) **(Logged values, so minus gives ratio)** 

$$  {Pooled\ Standard\  Deviation} = \sqrt\frac{(n_1-1)S_1^2 +(n_2-1)S_2^2}{(n_2 +n_2) -2}  $$  

$$  {Cohen's\ d\ value} = \frac{Enrichment}{Pooled\ Standard\ Deviation} $$

$$  {Correction\ Factor (J\ Factor)} = 1- \frac{3}{4df-1} $$

$$  {Hedges'\ g\ value} = Cohen's\ d\ \text{x}\ J\ $$

$$  {Variance\ in\ d (V_d)} = \frac{n_1- +n_2}{n_1 n_2} + \frac{d^2}{2(n_1 +n_2)}  $$

$$  {Variance\ in\ g (V_g)} = J^2\  \text{x}\ V_d  $$

$$  {Standard\ Error\ in\ g (SE_g)} = \sqrt{V_g}  $$

## Setup working environment and import data

In [1]:
import pandas as pd # Dataframes and file IO
import numpy as np # numerical calculations
%cd /Users/Ella1/Desktop/data sets 430AV2


/Users/Ella1/Desktop/data sets 430AV2


In [2]:
prefix = '430AV2_HypoT_9h_'   # define a prefix to add to column names (making indexing easier later)

In [3]:
# import the data file to a data frame 'df'
df=pd.read_table('DATASET-GSE6514.txt', delimiter='\t',  index_col=0) #,nrows=500)  
df.shape

(45101, 137)

In [4]:
# remove probes that are know to cross-hybridise to more than one target
df =df[~df.index.str.contains('_x_|_s_')]    #   important reverse selector ~ 
df.shape

(40569, 137)

## Look at column names and then setup filters for grouping columns into S and SD groups

In [5]:
df.columns

Index(['Symbol', 'Definition', 'Ensembl_id', 'Entrez_id', 'Unigene_id',
       'GO-Process', 'GO-Function', 'GO-Component', 'Pathway_info',
       'Putative microRNA binding sites',
       ...
       'adjp-HypoT_12hS_vs_HypoT_12hSD', 'GSM149636_HypoT_12hSD.CEL',
       'GSM149637_HypoT_12hSD.CEL', 'GSM149648_HypoT_12hSD.CEL',
       'GSM149649_HypoT_12hSD.CEL', 'GSM149650_HypoT_12hSD.CEL',
       'avg-HypoT_12hSD', 'ANOVA-rawp', 'ANOVA-adjp', 'largest fold'],
      dtype='object', length=137)

In [6]:
# define regular expressions for sleep (S) and sleep dep (SD) filters 
s_filt ='HypoT_9hS.CEL'
sd_filt ='HypoT_9hSD.CEL'

In [7]:
df_s=df.filter(regex= s_filt)
df_s.head()

,GSM149621_HypoT_9hS.CEL,GSM149622_HypoT_9hS.CEL,GSM149623_HypoT_9hS.CEL,GSM149624_HypoT_9hS.CEL,GSM149625_HypoT_9hS.CEL
Probesets,,,,,
1427138_at,6.62864,6.56007,6.38238,6.42859,6.50727
1425600_a_at,7.26288,6.29490,7.14532,7.05941,7.36697
1457168_at,5.79326,5.25913,6.13204,5.97069,5.92376
1450135_at,5.95135,5.54686,6.11248,5.60471,5.99862
1424014_at,7.47902,8.11068,7.58142,7.40510,7.57083


In [8]:
df_sd=df.filter(regex= sd_filt)
df_sd.head()

,GSM149626_HypoT_9hSD.CEL,GSM149627_HypoT_9hSD.CEL,GSM149628_HypoT_9hSD.CEL,GSM149629_HypoT_9hSD.CEL,GSM149630_HypoT_9hSD.CEL
Probesets,,,,,
1427138_at,6.54877,6.30270,6.49631,6.36580,6.39370
1425600_a_at,7.10842,6.64351,6.93398,7.33280,7.64352
1457168_at,5.56568,5.56388,5.38937,6.09373,6.00958
1450135_at,5.58607,5.60958,5.77467,6.15955,6.10577
1424014_at,8.39724,8.10581,8.26156,8.11123,7.37403


## Calculations 

In [9]:
# Enrichment

df[prefix+'Enrich'] = df.filter(regex=sd_filt).mean(axis=1) - df.filter(regex=s_filt).mean(axis=1)

In [10]:
df[prefix+'Enrich'].head()

Probesets
1427138_at     -0.079934
1425600_a_at    0.106550
1457168_at     -0.091328
1450135_at      0.004324
1424014_at      0.420564
Name: 430AV2_HypoT_9h_Enrich, dtype: float64

In [11]:
# Calculating Pooled StDev
Scount = df.filter(regex=s_filt).count(axis=1)
SDcount = df.filter(regex=sd_filt).count(axis=1)

StdevS = (Scount-1) * df.filter(regex=s_filt).var(axis=1)
StdevSD = (SDcount-1) * df.filter(regex=sd_filt).var(axis=1)

df[prefix+'poolStDev'] = np.sqrt((StdevS+StdevSD)/(Scount+ SDcount-2))

In [12]:
# Calculating Cohen's d
df[prefix+'Cohens_d'] = df[prefix+'Enrich'] / df[prefix+'poolStDev']

In [13]:
#df[prefix+'poolStDev'].head()
df[prefix+'Cohens_d'] .head()

Probesets
1427138_at     -0.804808
1425600_a_at    0.264054
1457168_at     -0.284054
1450135_at      0.016530
1424014_at      1.227290
Name: 430AV2_HypoT_9h_Cohens_d, dtype: float64

In [14]:
# Calculating J value (Correction factor)

df[prefix+'J'] = 1-(3/(4*(Scount+SDcount-1)))                              


In [15]:
# Calculating Hedge's g

df[prefix+'Hedges_g'] = df[prefix+'Cohens_d'] * df[prefix+'J']

In [16]:
#df[prefix+'J'].head()
df[prefix+'Hedges_g'] .head()

Probesets
1427138_at     -0.737741
1425600_a_at    0.242050
1457168_at     -0.260383
1450135_at      0.015153
1424014_at      1.125016
Name: 430AV2_HypoT_9h_Hedges_g, dtype: float64

In [17]:
# Calculating Var_d
Scount = df.filter(regex=s_filt).count(axis=1)
SDcount = df.filter(regex=sd_filt).count(axis=1)

Ftop1 = Scount + SDcount
Ftop2 = Scount * SDcount
Fbottom1 = np.square(df[prefix+'Cohens_d']) 
Fbottom2 =  2*(Scount + SDcount)


df[prefix+'Var_d'] = (Ftop1/Ftop2) + (Fbottom1 /Fbottom2)

In [18]:
#check output
df[prefix+'Var_d'].head()

Probesets
1427138_at      0.432386
1425600_a_at    0.403486
1457168_at      0.404034
1450135_at      0.400014
1424014_at      0.475312
Name: 430AV2_HypoT_9h_Var_d, dtype: float64

In [19]:
df[prefix+'Var_g'] = df[prefix+'Var_d'] * np.square(df[prefix+'J'])

In [20]:
# Calculating SEg
df[prefix+'SEg'] = np.sqrt(df[prefix+'Var_g'])

In [21]:
df.sort_values(by= '430AV2_HypoT_9h_Hedges_g', ascending=False, inplace=True)
df

,Symbol,Definition,Ensembl_id,Entrez_id,Unigene_id,GO-Process,GO-Function,GO-Component,Pathway_info,Putative microRNA binding sites,...,ANOVA-adjp,largest fold,430AV2_HypoT_9h_Enrich,430AV2_HypoT_9h_poolStDev,430AV2_HypoT_9h_Cohens_d,430AV2_HypoT_9h_J,430AV2_HypoT_9h_Hedges_g,430AV2_HypoT_9h_Var_d,430AV2_HypoT_9h_Var_g,430AV2_HypoT_9h_SEg
Probesets,,,,,,,,,,,,,,,,,,,,,
1418595_at,S3-12,perilipin 4,ENSMUSG00000002831,57435,NaN,NaN,NaN,lipid particle // cytoplasm // plasma membrane...,NaN,"mmu-miR-1(miRanda), mmu-miR-103(miRanda), mmu-...",...,3.895983e-23,1.981830,1.101802,0.159173,6.922062,0.916667,6.345224,2.795747,2.349204,1.532711
1428223_at,Mfsd2,major facilitator superfamily domain containin...,ENSMUSG00000028655,76574,NaN,transport,transporter activity,integral to membrane // membrane,NaN,"mmu-let-7a(RNAhybrid|miRanda), mmu-let-7b(RNAh...",...,3.915168e-14,1.188804,0.508870,0.080990,6.283145,0.916667,5.759550,2.373896,1.994732,1.412350
1426928_at,Ccdc93,coiled-coil domain containing 93,ENSMUSG00000026339,70829,NaN,NaN,NaN,NaN,NaN,"mmu-let-7(TargetScan), mmu-miR-100(RNAhybrid|m...",...,2.346228e-01,0.203020,0.125812,0.023572,5.337253,0.916667,4.892481,1.824313,1.532930,1.238115
1438341_at,LOC100047179 /// Tgm4,transglutaminase 4 (prostate) [Source:MGI Symb...,ENSMUSG00000025787,331046|100047179,NaN,peptide cross-linking,protein-glutamine gamma-glutamyltransferase ac...,NaN,NaN,"mmu-miR-149(miRanda), mmu-miR-193(miRanda), mm...",...,1.763192e-01,0.303312,0.229362,0.045031,5.093404,0.916667,4.668953,1.697138,1.426067,1.194181
1435119_at,NaN,NaN,NaN,NaN,Mm.395027,NaN,NaN,NaN,NaN,NaN,...,5.608550e-20,1.853314,1.045084,0.223649,4.672877,0.916667,4.283471,1.491789,1.253517,1.119606
1432629_at,NaN,predicted gene 9939 [Source:MGI Symbol;Acc:MGI...,ENSMUSG00000054247,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.262391e-01,0.434892,0.284690,0.062530,4.552848,0.916667,4.173444,1.436421,1.206993,1.098632
1460645_at,Chordc1,cysteine and histidine-rich domain (CHORD)-con...,ENSMUSG00000001774,66917,NaN,NaN,calcium ion binding // zinc ion binding,NaN,NaN,"mmu-miR-103(pictar), mmu-miR-107(pictar), mmu-...",...,3.282422e-12,1.619250,0.619044,0.137178,4.512703,0.916667,4.136644,1.418224,1.191702,1.091651
1443311_at,Thap4,THAP domain containing 4,NaN,67026,Mm.272356,NaN,nucleic acid binding // DNA binding // metal i...,NaN,NaN,NaN,...,3.729154e-03,0.244900,0.151470,0.034247,4.422827,0.916667,4.054258,1.378070,1.157961,1.076086
1435160_at,Ahsa2,"AHA1, activator of heat shock protein ATPase h...",ENSMUSG00000020288,268390,NaN,response to stress,ATPase activator activity // chaperone activat...,cytoplasm,NaN,"mmu-let-7a(miRanda), mmu-let-7b(miRanda), mmu-...",...,3.816544e-12,1.111020,0.346888,0.079561,4.360040,0.916667,3.996703,1.350497,1.134793,1.065267


In [22]:
df.columns

Index(['Symbol', 'Definition', 'Ensembl_id', 'Entrez_id', 'Unigene_id',
       'GO-Process', 'GO-Function', 'GO-Component', 'Pathway_info',
       'Putative microRNA binding sites',
       ...
       'ANOVA-adjp', 'largest fold', '430AV2_HypoT_9h_Enrich',
       '430AV2_HypoT_9h_poolStDev', '430AV2_HypoT_9h_Cohens_d',
       '430AV2_HypoT_9h_J', '430AV2_HypoT_9h_Hedges_g',
       '430AV2_HypoT_9h_Var_d', '430AV2_HypoT_9h_Var_g',
       '430AV2_HypoT_9h_SEg'],
      dtype='object', length=145)

### Import key file from BioMart and index probesets to MGI gene symbols

In [23]:
dfX=pd.read_table('../FHS project/Sleep notebook Copy/BioMart_Ensmbl_index/mart_export72_430v2430Av2.txt',index_col=[3])
 
dfX.pop('Affy mouse430 2 probeset') # remove 430V2 probeset info (not needed for 430AV2 indexing)
dfX.head(5)

,Ensembl Gene ID,Description,MGI symbol
Affy mouse430a 2 probeset,,,
1417126_a_at,ENSMUSG00000039221,ribosomal protein L22 like 1 [Source:MGI Symbo...,Rpl22l1
NaN,ENSMUSG00000095611,predicted gene 10597 [Source:MGI Symbol;Acc:MG...,Gm10597
1417730_at,ENSMUSG00000061731,exostoses (multiple) 1 [Source:MGI Symbol;Acc:...,Ext1
1417730_at,ENSMUSG00000061731,exostoses (multiple) 1 [Source:MGI Symbol;Acc:...,Ext1
NaN,ENSMUSG00000061731,exostoses (multiple) 1 [Source:MGI Symbol;Acc:...,Ext1


In [24]:
df_Join = df.join(dfX, how='left', sort=True)
df_FINAL1 = df_Join.groupby('MGI symbol').mean()
df_FINAL1[df_FINAL1.index.duplicated()==True]   # checking that no duplicate entries exist in the dataframe

,GSM149516_CerCx_3hS.CEL,GSM149517_CerCx_3hS.CEL,GSM149518_CerCx_3hS.CEL,GSM149519_CerCx_3hS.CEL,GSM149520_CerCx_3hS.CEL,avg-CerCx_3hS,log_fold-CerCx_3hS_vs_CerCx_3hSD,fold-CerCx_3hS_vs_CerCx_3hSD,rawp-CerCx_3hS_vs_CerCx_3hSD,adjp-CerCx_3hS_vs_CerCx_3hSD,...,ANOVA-adjp,largest fold,430AV2_HypoT_9h_Enrich,430AV2_HypoT_9h_poolStDev,430AV2_HypoT_9h_Cohens_d,430AV2_HypoT_9h_J,430AV2_HypoT_9h_Hedges_g,430AV2_HypoT_9h_Var_d,430AV2_HypoT_9h_Var_g,430AV2_HypoT_9h_SEg
MGI symbol,,,,,,,,,,,,,,,,,,,,,


### Columns from the list above can then easily be picked to produce files for use later. Examples below given:
 #### df3 = average S and SD expression for the platform and the log-fold changes
 #### df4 = Hedges g  values and associated variance for Meta-analysis (after indexing)

In [25]:
# df3 = df_FINAL1.loc[:,[u'avg-SD', u'avg-S', u'log_fold-S_vs_SD']]
# df3.columns =[prefix+'avg-SD', prefix+'avg-S', prefix+'log_fold-S_vs_SD']
# df3.to_csv('input_files/430AV2_SymbolExpression_forIndex.csv')

In [26]:
df4 = df_FINAL1.loc[:,[u'430AV2_HypoT_9h_Enrich',u'430AV2_HypoT_9h_Hedges_g', u'430AV2_HypoT_9h_Var_g', u'430AV2_HypoT_9h_SEg']]
df4.to_csv('../FHS project/Sleep notebook Copy/IPython_notebooks/input_files/430AV2_HypoT_9h_SymbolforIndexHedges.csv')

In [27]:
df4.head(10)  # check final ouput

,430AV2_HypoT_9h_Enrich,430AV2_HypoT_9h_Hedges_g,430AV2_HypoT_9h_Var_g,430AV2_HypoT_9h_SEg
MGI symbol,,,,
0610005C13Rik,0.052672,0.668878,0.358481,0.598733
0610008F07Rik,0.097044,0.490025,0.348117,0.590015
0610009B22Rik,-0.094638,-0.472425,0.347270,0.589297
0610009D07Rik,-0.163790,-0.359639,0.342656,0.585367
0610009O20Rik,-0.193852,-0.707710,0.361154,0.600961
0610010K14Rik,-0.040696,-0.086165,0.336482,0.580071
0610012G03Rik,-0.159512,-0.422543,0.374148,0.611507
0610031J06Rik,-0.106980,-0.612502,0.354869,0.595709
0610037L13Rik,-0.094966,-0.671653,0.358667,0.598888
